# Time series analyzer
## A Climate Asset Pricing model -- AR6 analysis notebook

In this notebook, we provide a smooth framework for users to analyze the output of CAP6. This is meant to be generic analysis, but can be used to make nicer figures; we use it here to make the preferred and featured runs figure in the paper. 

In [ ]:
import sys
import datetime

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import matplotlib as mpl
import seaborn as sns

from src.tree import TreeModel
from src.climate import BPWClimate
from src.emit_baseline import BPWEmissionBaseline
from src.damage import BPWDamage
from src.tools import import_csv
from src.analysis.output_unpacker import OutputUnpacker
from src.analysis.tree_diagrams import TreeDiagram

# plotting parameters
color_list = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7'] * 2
marker_list = ['o', 's', 'P', '+', 'D', 'v', '3'] * 2
markersize = 6
linestyle_list = ['solid', 'dashed', 'dashdot', 'dotted'] * 4
linewidth = 2
node_times = [2020, 2030, 2060, 2100, 2150, 2200]
x_label = 'Year'

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
cdds_params={'axes.linewidth': 3,
 'axes.axisbelow': False,
 'axes.edgecolor': 'black',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'black',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'axes.titlesize': 20,
 'axes.labelsize': 20,
 'axes.titlelocation': 'left',
 'figure.facecolor': 'white',
 'figure.figsize': (18, 10),
 'lines.solid_capstyle': 'round',
 'lines.linewidth': 2.5,
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'black',
 'legend.frameon': False,
 'xtick.bottom': True,
 'xtick.major.width': 3,
 'xtick.major.size': 6,
 'xtick.color': 'black',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'black',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.right': False,
 'ytick.color' : 'black',
 'ytick.major.width': 3, 
 'ytick.major.size': 6,
 'axes.prop_cycle': plt.cycler(linestyle=linestyle_list),
 'font.size': 16,
 'font.family': 'serif'}
plt.rcParams.update(cdds_params)

# make base filename
today = datetime.datetime.now()
year = str(today.year)
day = str(today.day)
month = str(today.month)

basefile = ''.join([month, '-', day, '-', year, '-'])

## Make lists of necessary model classes

Make sure this matches the main file! To do this properly, the right classes with the right model parameters need to be initiated. 

In [ ]:
"""Import header, indices, and data from reference .csv file.
"""

data_csv_file = 'BPW_research_runs'
header, indices, data = import_csv(data_csv_file, delimiter=',', indices=2)

"""Run TCREZClimate for runs of interest. To change which runs the code does,
change desired_runs.
"""

# runs needing analysis
done_runs = [0,1,2,3] # main runs

tree_list = []
damage_list = []
climate_list = []
emit_baseline_list = []

for i in done_runs:
    ra, eis, pref, growth, tech_chg, tech_scale, dam_func,\
            baseline_num, tip_on, bs_premium, d_unc, t_unc, nfl = data[i]

    baseline_num = int(baseline_num)
    dam_func = int(dam_func)
    tip_on = int(tip_on)
    d_unc = int(d_unc)
    t_unc = int(t_unc)
    nfl = int(nfl)

    """Initialize model classes. First is the tree model.
    """

    tmp_tree = TreeModel(decision_times=[0, 10, 40, 80, 130, 180, 230],
                  prob_scale=1.0)

    """Emission baseline model. We also run its setup function.
    """

    tmp_baseline_emission_model = BPWEmissionBaseline(tree=tmp_tree,
                                                  baseline_num=baseline_num)
    tmp_baseline_emission_model.baseline_emission_setup()

    """Climate class. We set draws to the number of Monte Carlo samples to take
    from damage distributions, such as TCRE, and if the floor is on.
    """

    draws = 3 * 10**6
    tmp_climate = BPWClimate(tmp_tree, tmp_baseline_emission_model, draws=draws)

    """Damage class. We set draws to the number of Monte Carlo samples to take
    from damage distributions, such as TCRE. We also pass a list of constant
    values of mitigation for the damage simulation.
    """

    d_m = 0.01
    mitigation_constants = np.arange(0, 1 + d_m, d_m)[::-1]
    tmp_df = BPWDamage(tree=tmp_tree, emit_baseline=tmp_baseline_emission_model,
                   climate=tmp_climate, mitigation_constants=mitigation_constants,
                   draws=draws)

    """Run the damage simulation. If a damage simulation has already been run,
    then comment the line below and uncomment the df.import_damages() line.

    NOTE: import_damages has the optional argument for a filename. This is
    important if you're varying the damage function across simulations.

    The default is currently "BPW_simulated_damages_TCRE.csv".
    """

    damsim_filename = ''.join(["BPW_simulated_damages_df", str(dam_func),
                               "_TP", str(tip_on), "_SSP", str(baseline_num),
                               "_dunc", str(d_unc), "_tunc", str(t_unc)])

    tmp_df.import_damages(file_name=damsim_filename)
    
    tree_list.append(tmp_tree)
    emit_baseline_list.append(tmp_baseline_emission_model)
    climate_list.append(tmp_climate)
    damage_list.append(tmp_df)

## Make list of files of interest

Here we make lists of file prefixes (i.e., whatever the `name` parameter was when the simulation was executed) and paths. We take the list of paths and prefixes and make lists of filenames, which will be passed to `OutputUnpacker` later.

In [ ]:
# list of prefixes to output files of EZClimate. i.e., the string before "_node_period_output.csv".
# in EZClimate main file, this is the parameter "name" located at about line 42 

# prefixes ("name" variable in main file)
prefix_list = ['pref-run', 'feat-15', 'feat-25', 'feat-3']

# path to data, in order of the prefixes listed above
path_list = ['/data/keeling/a/adammb4/ClimateEcon/ez-climate/TCREZClimate/data/'] * len(prefix_list) 

# file description - this will be used in plot titles and helps keep track of which file is which
# in the prefix_list
descriptions = ['2% discount rate', '1.5% discount rate', '2.5% discount rate', '3% discount rate']

# make list of file names to be unpacked
output_list = []
picklefile_list = []

# total files
N_files = len(prefix_list)

# populate output_list and picklefile_list
for file in range(0, N_files):
    tmp_filename = path_list[file] + prefix_list[file] + "_node_period_output.csv"
    tmp_picklefile_name = path_list[file] + prefix_list[file] + "_log.pickle"
    output_list.append(tmp_filename)
    picklefile_list.append(tmp_picklefile_name)

In [ ]:
# use OutputUnpacker to "unpack" the files above

# make one array of objects which correspond to the 'output' files (i.e., the files that end with
# _node_period_output.csv) and another for the pickle files
output_obj_list = []
pickle_obj_list = []

# populate these arrays
for file in range(0, N_files):
    tmp_output_obj = OutputUnpacker(output_list[file], descriptions[file], 'output', 
                                    tree=tree_list[file], emit_baseline=emit_baseline_list[file],
                                    climate=climate_list[file], damage=damage_list[file])
    output_obj_list.append(tmp_output_obj)
    tmp_pickle_obj = OutputUnpacker(picklefile_list[file], descriptions[file], 'pickle',
                                    tree=tree_list[file], emit_baseline=emit_baseline_list[file],
                                    climate=climate_list[file], damage=damage_list[file])
    pickle_obj_list.append(tmp_pickle_obj)

## SAVE FIGS?!

In [ ]:
save_fig = False
fig_filename_prefix = 'ra-impact'

## Time series plots

Here we plot a number of time series of key model variables, such as:
- average mitigation
- expected mitigation
- average temperature
- average emissions
- cost of carbon
- CO$_2$ concentrations
- economic damages

### Cost of carbon

In [ ]:
fig, ax = plt.subplots(1, figsize=(9.5,7))

for file in range(0, N_files):
    ax.plot(node_times, np.mean(output_obj_list[file].price_path, axis=0), 
                  color=color_list[file], marker=None, 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)
    print(np.mean(output_obj_list[file].price_path, axis=0)[0])


ax.set_ylabel("Carbon price ($ USD 2020)")
ax.legend()
ax.set_xlabel(x_label)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylim((0,250))
ax.set_yticks([0,50, 100,150, 200, 250])
ax.set_xticks([2020, 2100, 2200])
sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, fig_filename_prefix, '-cost.png']), dpi=400)

### Average mitigation

In [ ]:
fig, ax = plt.subplots(1)

for file in range(0, N_files):
    ax.plot(node_times, np.mean(pickle_obj_list[file].m_opt_path, axis=0), 
            color=color_list[file], marker=marker_list[file], 
            label=pickle_obj_list[file].description, markersize=markersize,
            linestyle=linestyle_list[file], linewidth=linewidth)

ax.set_ylabel("Average mitigation")
ax.legend()
ax.set_xlabel(x_label)
ax.tick_params(axis='both')
#ax.axhline(1, color='g', linestyle='dashed')
ax.set_ylim((0,1.6))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if save_fig:
    fig.savefig(''.join([basefile, fig_filename_prefix, '-mit.png']), dpi=400)

### Expected mitigation

Difference from average, as this is the weighted average of mitigation

In [ ]:
fig, ax = plt.subplots()

for file in range(0, N_files):
    ax.plot(node_times, np.mean(output_obj_list[file].exp_mit_path, axis=0), 
            color=color_list[file], marker=marker_list[file], 
            label=pickle_obj_list[file].description, markersize=markersize,
            linestyle=linestyle_list[file], linewidth=linewidth)

ax.set_ylabel("Expected mitigation")
ax.legend()
ax.set_xlabel(x_label)
#ax.axhline(1, color='g', linestyle='dashed')
ax.set_ylim((0,1.3))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if save_fig:
    fig.savefig(''.join([basefile, fig_filename_prefix, '-exp-mit.png']), dpi=400)

### (Mitigated) emissions

In [ ]:
fig, ax = plt.subplots()

ax.plot(tmp_baseline_emission_model.times[:-5], tmp_baseline_emission_model.baseline_gtco2[:-5], 
        color='slategrey', label="SSP2")
ax.plot(tmp_baseline_emission_model.times[:-5], (1 - 0.6) * tmp_baseline_emission_model.baseline_gtco2[:-5], 
        color='slategrey', label="60% Mitigation", linestyle="dashed")
ax.plot(tmp_baseline_emission_model.times[:-5], (1 - 0.9) * tmp_baseline_emission_model.baseline_gtco2[:-5], 
        color='slategrey', label="90% Mitigation", linestyle="dashdot")

for file in range(0, N_files):
    ax.plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].emis_gtco2_full, axis=0), 
                  color=color_list[file], marker=marker_list[file], 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)

    
ax.set_ylabel("Average emissions (GtCO$_2$ yr$^{-1}$)")
ax.set_xlabel(x_label)
ax.legend()

### Average temperature

In [ ]:
fig, ax = plt.subplots()

for file in range(0, N_files):
    ax.plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].temp_full, axis=0), 
                  color=color_list[file], marker=marker_list[file], 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)

    
ax.set_ylabel("Average temperature anomaly (K)")
ax.legend()
ax.set_xlabel(x_label)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.set_ylim((0,5))

if save_fig:
    fig.savefig(''.join([basefile, fig_filename_prefix, '-temp.png']), dpi=400)

In [ ]:
fig, ax = plt.subplots(1)

for file in range(0, N_files):
    ax.plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].temp_full, axis=0) - np.mean(pickle_obj_list[file].temp_full, axis=0)[0], 
                  color=color_list[file], marker=marker_list[file], 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)

    
ax.set_ylabel("Average temperature anomaly from 2020 (K)")
ax.legend()
ax.set_xlabel(x_label)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

### CO$_2$ concentrations

In [ ]:
fig, ax = plt.subplots()

for file in range(0, N_files):
    ax.plot(node_times, np.mean(output_obj_list[file].ghg_lvl_path, axis=0), 
                  color=color_list[file], marker=marker_list[file], 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)


ax.set_ylabel("Average CO$_2$ concentrations (ppm)")
ax.legend()
ax.set_xlabel(x_label)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if save_fig:
    fig.savefig(''.join([basefile, fig_filename_prefix, '-co2.png']), dpi=400)

### Economic damages

In [ ]:
fig, ax = plt.subplots()

for file in range(0, N_files):
    ax.plot(node_times, np.mean(pickle_obj_list[file].econ_dam_path, axis=0), 
                  color=color_list[file], marker=marker_list[file], 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)

ax.set_ylabel("Average economic damages (% GDP)")
ax.legend()
ax.set_xlabel(x_label)


In [ ]:
fig, ax = plt.subplots()

for file in range(0, N_files):
    ax.plot(np.mean(pickle_obj_list[file].temp_path, axis=0), np.mean(pickle_obj_list[file].econ_dam_path, axis=0), 
                  color=color_list[file], marker=marker_list[file], 
                  label=pickle_obj_list[file].description, markersize=markersize,
                  linestyle=linestyle_list[file], linewidth=linewidth)

ax.set_ylabel("Average economic damages (% GDP)")
ax.legend()
ax.set_xlabel("Temperature (K)")

## 6 Panels -- Main figure in our paper

In [ ]:
import matplotlib.transforms as mtransforms

fig, ax = plt.subplot_mosaic([['A', 'B', 'C'], ['D', 'E', 'F']], sharex=True,
                             gridspec_kw={'height_ratios': [1, 1], 'width_ratios': [1, 1, 1]},
                            figsize=(22,10))

pref_lw = 3.75
feat_lw = 2
pref, = ax['A'].plot(node_times, np.mean(output_obj_list[0].price_path, axis=0), 
              color=color_list[0], 
              label=pickle_obj_list[0].description,
              linestyle=linestyle_list[0], linewidth=pref_lw)
feat15, = ax['A'].plot(node_times, np.mean(output_obj_list[1].price_path, axis=0), 
              color=color_list[1], 
              label=pickle_obj_list[1].description,
              linestyle=linestyle_list[1], linewidth=linewidth)
feat25, = ax['A'].plot(node_times, np.mean(output_obj_list[2].price_path, axis=0), 
              color=color_list[2], 
              label=pickle_obj_list[2].description,
              linestyle=linestyle_list[2], linewidth=linewidth)
feat3, = ax['A'].plot(node_times, np.mean(output_obj_list[3].price_path, axis=0), 
              color=color_list[3], 
              label=pickle_obj_list[3].description,
              linestyle=linestyle_list[3], linewidth=linewidth)

#ax['B'].hlines(1.5, 2020, 2200, alpha=0.5, color='grey', linestyle='dashed', linewidth=1.75)
#ax['B'].vlines(2100, 1, 2, alpha=0.5, color='grey', linestyle='dashed', linewidth=1.75)

# other plots
for file in range(0, N_files):
    if file == 0:
        ax['B'].plot(node_times, np.mean(output_obj_list[file].exp_mit_path, axis=0) * 100, 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=pref_lw)
        
        ax['C'].plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].emis_gtco2_full, axis=0), 
                      color=color_list[file], 
                      linestyle=linestyle_list[file], linewidth=pref_lw)
        
        ax['D'].plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].temp_full, axis=0), 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=pref_lw)

        ax['E'].plot(node_times, np.mean(output_obj_list[file].ghg_lvl_path, axis=0), 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=pref_lw)

        ax['F'].plot(node_times, np.mean(pickle_obj_list[file].econ_dam_path * 100, axis=0), 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=pref_lw)
        
    else:
        ax['B'].plot(node_times, np.mean(output_obj_list[file].exp_mit_path, axis=0) * 100, 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=feat_lw)
        
        ax['C'].plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].emis_gtco2_full, axis=0), 
                      color=color_list[file], 
                      linestyle=linestyle_list[file], linewidth=feat_lw)
        
        ax['D'].plot(tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[file].temp_full, axis=0), 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=feat_lw)

        ax['E'].plot(node_times, np.mean(output_obj_list[file].ghg_lvl_path, axis=0), 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=feat_lw)

        ax['F'].plot(node_times, np.mean(pickle_obj_list[file].econ_dam_path * 100, axis=0), 
                      color=color_list[file], 
                      label=pickle_obj_list[file].description,
                      linestyle=linestyle_list[file], linewidth=feat_lw)
        
        
# plot SSP2 for clarity
ax['C'].plot(tmp_baseline_emission_model.times[:-5], tmp_baseline_emission_model.baseline_gtco2[:-5], 
        color='slategrey', label="SSP2", linewidth=feat_lw)
ax['C'].legend(loc='right', fontsize=19)

panels = ['A', 'B', 'C', 'D', 'E', 'F']
for panel in panels:
    ax[panel].set_xticks([2020, 2100, 2200])

# set ticks
ax['A'].set_yticks([0, 200, 400, 600])
ax['B'].set_yticks([0, 20, 40, 60, 80, 100])
ax['C'].set_yticks([0, 10, 20, 30, 40, 50])
ax['D'].set_yticks([1, 1.5, 2, 2.5])
ax['E'].set_yticks([350, 400, 450, 500, 550])
ax['F'].set_yticks([0, 2, 4, 6])

# set labels
ax['A'].set_ylabel("Cost (2020 $USD)", fontsize=19)
ax['B'].set_ylabel("Expected abatement (%)", fontsize=19)
ax['C'].set_ylabel("Emissions (GtCO$_2$ yr$^{-1}$)", fontsize=19)
ax['D'].set_ylabel("Temperature ($^{\circ}$C)", fontsize=19)
ax['E'].set_ylabel("CO$_2$ concentration (ppm)", fontsize=19)
ax['F'].set_ylabel("Econ. Damage (% GDP)", fontsize=19)

ax['D'].set_xlabel("Year", fontsize=19)
ax['E'].set_xlabel("Year", fontsize=19)
ax['F'].set_xlabel("Year", fontsize=19)
#ax['A'].set_ylim((0,310))

# bottom legend
fig.legend([feat15, pref, feat25, feat3], ["1.5% discount rate", "2% discount rate", "2.5% discount rate", "3% discount rate"], 
           bbox_to_anchor=(0.48, -0.045), 
           loc='lower center', ncol=4, fancybox=True, shadow=True, fontsize=18)

# add labels
right = ['A', 'B', 'C', 'D', 'E', 'F']
for label in right:
    # label physical distance in and down:
    trans = mtransforms.ScaledTranslation(0, 0.0, fig.dpi_scale_trans)
    ax[label].text(0.9, 1.0, label, transform=ax[label].transAxes + trans, fontsize=22, fontweight='bold',
            verticalalignment='top', bbox=dict(facecolor='none', edgecolor='none', pad=1))
    
sns.despine(trim=True, offset=10)

#fig.savefig(''.join([basefile, 'pref-runs-NFL.png']),bbox_inches='tight', dpi=400)

### Print some relevant values (temperature and so on)

In [ ]:
for i in range(len(pickle_obj_list)):
    print(np.mean(pickle_obj_list[i].econ_dam_path, axis=0) * 100)

In [ ]:
for i in range(len(pickle_obj_list)):
    print(np.mean(pickle_obj_list[i].temp_full, axis=0)[0], 
         np.mean(pickle_obj_list[i].temp_full, axis=0)[1],
         np.mean(pickle_obj_list[i].temp_full, axis=0)[4],
         np.mean(pickle_obj_list[i].temp_full, axis=0)[8],
         np.mean(pickle_obj_list[i].temp_full, axis=0)[13],
         np.mean(pickle_obj_list[i].temp_full, axis=0)[18])

In [ ]:
tmp_baseline_emission_model.times[:-5], np.mean(pickle_obj_list[0].emis_gtco2_full, axis=0)